## Problem Statement

#### Your client is a Financial Distribution company. Over the last 10 years, they have created an offline distribution channel across country. They sell Financial products to consumers by hiring agents in their network. These agents are freelancers and get commission when they make a product sale.



Overview of your client On-boarding process
The Managers at your client are primarily responsible for recruiting agents. Once a manager has identified a potential applicant, the would explain the business opportunity to the agent. Once the agent provides the consent, an application is made to your client to become an agent. This date is known as application_receipt_date.

In the next 3 months, this potential agent has to undergo a 7 days training at the your client's branch (about Sales processes and various products) and clear a subsequent examination in order to become an agent.




The problem - Who are the best agents?
As is obvious in the above process, there is a significant investment which your cleint makes in identifying, training and recruiting these agents. However, there are a set of agents who do not bring in the expected resultant business.

Your client is looking for help from data scientists like you to help them provide insigths using their past recruitment data. They want to predict the target variable for each potential agent, which would help them identify the right agents to hire.


### Data Ditictionary

In [1]:
import tensorflow as tf
tf.VERSION

D:\Study\Great Lakes\INtro to python\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Study\Great Lakes\INtro to python\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Study\Great Lakes\INtro to python\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Study\Great Lakes\INtro to python\Anacon

'1.14.0'

In [3]:
!pip install tensorflow==2.3

In [ ]:
import numpy as np
import pandas as pd 
import os
from sklearn.model_selection import KFold,StratifiedKFold
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight,compute_sample_weight
from keras.layers import Dense,Dropout
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
path='D:/Study/Hackathon analytics vidhya/Hackathon AV hiring  10092020'
train_df=pd.read_csv(os.path.join(path,'train.csv'))
test_df=pd.read_csv(os.path.join(path,'test.csv'))
submission_df=pd.read_csv(os.path.join(path,'sample.csv'))

In [ ]:
train_df['Business_Sourced'].value_counts(normalize = True)*100

In [ ]:
train_df.info()

In [ ]:
#train_df=train_df.drop_duplicates(subset=[ele for ele in list(train_df.columns) if ele not in ['ID']])
#Adding more Features
combine_set=pd.concat([train_df,test_df],axis=0)

In [ ]:
#from sklearn.preprocessing import LabelEncoder
#le=LabelEncoder()


combine_set['Applicant_City_PIN']=combine_set['Applicant_City_PIN'].fillna((combine_set['Applicant_City_PIN'].median())).astype('int')
combine_set['Applicant_Gender'].fillna(combine_set['Applicant_Gender'].mode().iloc[0],inplace=True)
combine_set['Applicant_Marital_Status'].fillna(combine_set['Applicant_Marital_Status'].mode().iloc[0],inplace=True)
combine_set['Applicant_Occupation'].fillna('Missing',inplace=True)
combine_set['Applicant_Qualification'].fillna(combine_set['Applicant_Qualification'].mode().iloc[0],inplace=True)

combine_set['Manager_Joining_Designation'].fillna(combine_set['Manager_Joining_Designation'].mode().iloc[0],inplace=True)
combine_set['Manager_Current_Designation'].fillna(combine_set['Manager_Current_Designation'].mode().iloc[0],inplace=True)
combine_set['Manager_Grade'].fillna(combine_set['Manager_Grade'].median(),inplace=True)
combine_set['Manager_Status'].fillna(combine_set['Manager_Status'].mode().iloc[0],inplace=True)
combine_set['Manager_Gender'].fillna(combine_set['Manager_Gender'].mode().iloc[0],inplace=True)
combine_set['Manager_Num_Application'].fillna(combine_set['Manager_Num_Application'].median(),inplace=True)
combine_set['Manager_Num_Coded'].fillna(combine_set['Manager_Num_Coded'].median(),inplace=True)
combine_set['Manager_Business'].fillna(combine_set['Manager_Business'].median(),inplace=True)
combine_set['Manager_Num_Products'].fillna(combine_set['Manager_Num_Products'].median(),inplace=True)
combine_set['Manager_Business2'].fillna(combine_set['Manager_Business2'].median(),inplace=True)
combine_set['Manager_Num_Products2'].fillna(combine_set['Manager_Num_Products2'].median(),inplace=True)

from datetime import datetime, date
ref_date = datetime.now()
# Date converion 
combine_set[["Application_Receipt_Date", "Applicant_BirthDate", "Manager_DOJ",'Manager_DoB']] = combine_set[["Application_Receipt_Date", "Applicant_BirthDate", "Manager_DOJ",'Manager_DoB']].apply(pd.to_datetime,errors ='coerce')

# reciopt count freq encoding
receipt_set=combine_set['Application_Receipt_Date'].value_counts().to_dict()
combine_set['Application_Receipt_Date_Count']=combine_set['Application_Receipt_Date'].map(receipt_set)

# NAT replacement
combine_set['Applicant_BirthDate'] =  combine_set['Applicant_BirthDate'].astype(str)
combine_set['Applicant_BirthDate'] = combine_set['Applicant_BirthDate'].apply(lambda x : ref_date if x=="NaT" else x)

combine_set['Manager_DOJ'] =  combine_set['Manager_DOJ'].astype(str)
combine_set['Manager_DOJ'] = combine_set['Manager_DOJ'].apply(lambda x : ref_date if x=="NaT" else x)

combine_set['Manager_DoB'] =  combine_set['Manager_DoB'].astype(str)
combine_set['Manager_DoB'] = combine_set['Manager_DoB'].apply(lambda x : ref_date if x=="NaT" else x)

# Age Calculation
combine_set['Applicant_Age'] = combine_set['Applicant_BirthDate'].apply(lambda x: len(pd.date_range(start = x, end = ref_date, freq = 'Y')))
combine_set['Manager_Exp'] = combine_set['Manager_DOJ'].apply(lambda x: len(pd.date_range(start = x, end = ref_date, freq = 'Y')))
combine_set['Manager_Age'] = combine_set['Manager_DoB'].apply(lambda x: len(pd.date_range(start = x, end = ref_date, freq = 'Y')))

# Applicant age , manager experience and manager age feature imputed with median 
combine_set['Applicant_Age'].fillna(combine_set['Manager_Exp'].median(),inplace=True)
combine_set['Manager_Exp'].fillna(combine_set['Manager_Exp'].median(),inplace=True)
combine_set['Manager_Age'].fillna(combine_set['Manager_Age'].median(),inplace=True)

#Droping date columns
col = ["Application_Receipt_Date", "Applicant_BirthDate", "Manager_DOJ",'Manager_DoB']
combine_set.drop(col, axis=1, inplace=True)


combine_set1 = combine_set.copy()
# Introducing two columns
combine_set1['Manager_Actual_Business_ratio']=combine_set1['Manager_Business']-combine_set1['Manager_Business2']
combine_set1['Manager_Num_Coded_ratio']=combine_set1['Manager_Num_Coded']-combine_set1['Manager_Num_Products2']
combine_set1['Manager_Actual_Business_ratio'].fillna(combine_set1['Manager_Actual_Business_ratio'].median(),inplace=True)
combine_set1['Manager_Num_Coded_ratio'].fillna(combine_set1['Manager_Num_Coded_ratio'].median(),inplace=True)

# Applicant_City_PIN feature imputed with median 
combine_set1['Applicant_City_PIN'] = combine_set1['Applicant_City_PIN'].astype(str).str[:-5].astype(np.int64)

# Applicant_City_PIN feature imputed with median
combine_set1['Office_PIN'] = combine_set1['Office_PIN'].astype(str).str[:-5].astype(np.int64)


In [ ]:
"""
# Manager_Joining_Designation feature imputed with mode & converted to categorical 
combine_set['Manager_Joining_Designation']=combine_set['Manager_Joining_Designation'].fillna('Designation_Unkown')
combine_set['Manager_Joining_Designation']=le.fit_transform(combine_set['Manager_Joining_Designation'])

# Manager_Joining_Designation feature imputed with mode & converted to categorical 
combine_set['Manager_Current_Designation']=combine_set['Manager_Current_Designation'].fillna('Current_Designation_Unkown')
combine_set['Manager_Current_Designation']=le.fit_transform(combine_set['Manager_Current_Designation'])

# Manager_Grade feature imputed with median 
combine_set['Manager_Grade']=le.fit_transform(combine_set['Manager_Grade'])

# Manager_Joining_Designation feature imputed with mode & converted to categorical 
combine_set['Manager_Status']=combine_set['Manager_Status'].fillna('Manager_Status_Unkown')
combine_set['Manager_Status']=le.fit_transform(combine_set['Manager_Status'])

# Manager_Joining_Designation feature imputed with mode & converted to categorical 
combine_set['Manager_Status']=combine_set['Manager_Status'].fillna('Manager_Status_Unkown')
combine_set['Manager_Status']=le.fit_transform(combine_set['Manager_Status'])

# Manager_Grade feature imputed with median 
combine_set['Manager_Num_Application']=le.fit_transform(combine_set['Manager_Num_Application'])

# Manager_Num_Coded feature imputed with median 
combine_set['Manager_Num_Coded']=le.fit_transform(combine_set['Manager_Num_Coded'])

# Manager_Business feature imputed with median 
combine_set['Manager_Business']=le.fit_transform(combine_set['Manager_Business'])

# Manager_Num_Products feature imputed with median 
combine_set['Manager_Num_Products']=le.fit_transform(combine_set['Manager_Num_Products'])

# Manager_Business2 feature imputed with median 
combine_set['Manager_Business2']=le.fit_transform(combine_set['Manager_Business2'])

# Manager_Num_Products2 feature imputed with median 
combine_set['Manager_Num_Products2']=le.fit_transform(combine_set['Manager_Num_Products2'])
"""

"""
# numeric columns
>>> df.select_dtypes(include='float').fillna(\
     df.select_dtypes(include='float').mean().iloc[0],\                    
     inplace=True)

# categorical columns
>>> df.select_dtypes(include='object').fillna(\
 ...: df.select_dtypes(include='object').mode().iloc[0])
"""


In [ ]:
combine_set1.head()

In [ ]:
combine_set1.info()

### OHE and le encoding

In [ ]:
#Encoding Categorical Columns
le=LabelEncoder()
for col in combine_set1.select_dtypes(include='object').columns:
    if col not in ['Business_Sourced']:
#         fe=combine_set.groupby([col]).size()/len(combine_set)
#         combine_set[col]=combine_set[col].apply(lambda x: fe[x])
        df=pd.get_dummies(combine_set1[col],drop_first=True)
        combine_set1=pd.concat([combine_set1,df],axis=1).drop([col],axis=1)
          
    elif col!='Business_Sourced':
        combine_set[col]=le.fit_transform(combine_set[col].astype(str))
    else:
        pass
        
combine_set1.head(5) 

In [ ]:
X=combine_set1[combine_set1['Business_Sourced'].isnull()==False].drop(['Business_Sourced'],axis=1)
y=le.fit_transform(combine_set1[combine_set1['Business_Sourced'].isnull()==False]['Business_Sourced'])
y=pd.DataFrame(y,columns=['Business_Sourced'])
X_main_test=combine_set1[combine_set1['Business_Sourced'].isnull()==True].drop(['Business_Sourced'],axis=1)


In [ ]:
combine_set1.to_csv("combine_set1_train_test.csv",index=False)

In [ ]:
combine_set1.shape

## # **Standardizing all Features**

In [ ]:
y_hat=to_categorical(y)
y_hat=pd.DataFrame(y_hat)
sc_X=StandardScaler()
X=sc_X.fit_transform(X)
X=pd.DataFrame(X)
sc_X_main=StandardScaler()
X_main_test=sc_X_main.fit_transform(X_main_test)
X.head(5)

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X,y_hat,test_size=0.2,random_state=294)

## **Sequential NN**

In [ ]:
classifier=Sequential()

classifier.add(Dense(512,activation='relu', kernel_initializer='uniform',input_shape=(X_train.shape[1],)))
classifier.add(Dropout(0.2))
classifier.add(Dense(256,activation='relu',kernel_initializer='uniform'))
classifier.add(Dense(200,activation='relu',kernel_initializer='uniform'))
classifier.add(Dense(128,activation='relu',kernel_initializer='uniform'))
classifier.add(Dense(64,activation='relu',kernel_initializer='uniform'))
classifier.add(Dense(32,activation='relu',kernel_initializer='uniform'))
classifier.add(Dense(11,activation='softmax'))

classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
callback_lr=ReduceLROnPlateau(monitor='val_loss',patience=3,factor=0.3,min_lr=0.00001)
callback_mc=ModelCheckpoint(filepath='model_repli.hdf5',monitor='val_accuracy',save_best_only=True,mode='max')

classifier.fit(X_train,y_train,epochs=50,batch_size=32,validation_data=(X_val,y_val),callbacks=[callback_lr,callback_mc])

classifier=load_model('model_repli.hdf5')
pred_val=classifier.predict(X_val)

preds=classifier.predict(X_main_test)

In [ ]:
# classifier=Sequential()

# classifier.add(Dense(512,activation='relu', kernel_initializer='uniform',input_shape=(X_train.shape[1],)))
# classifier.add(Dropout(0.2))
# classifier.add(Dense(256,activation='relu',kernel_initializer='uniform'))
# # classifier.add(Dense(200,activation='relu',kernel_initializer='uniform'))
# classifier.add(Dense(128,activation='relu',kernel_initializer='uniform'))
# classifier.add(Dense(128,activation='relu',kernel_initializer='uniform'))
# # classifier.add(Dense(32,activation='relu',kernel_initializer='uniform'))
# classifier.add(Dense(11,activation='softmax'))

# classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# callback_lr=ReduceLROnPlateau(monitor='val_loss',patience=3,factor=0.5,min_lr=0.00001)
# callback_mc=ModelCheckpoint(filepath='model_reli2.hdf5',monitor='val_accuracy',save_best_only=True,mode='max')

# classifier.fit(X_train,y_train,epochs=50,batch_size=32,validation_data=(X_val,y_val),callbacks=[callback_lr,callback_mc])

# classifier=load_model('model_reli2.hdf5')
# preds2_val=classifier.pedict(X_val)

# preds2=classifier.predict(X_main_test)

### # **KFOLD (LGBM)**

In [ ]:

kf=KFold(n_splits=10,shuffle=True,random_state=2020)
# sc_X=StandardScaler()
# X=pd.DataFrame(sc_X.fit_transform(X))
preds={}
acc_score=0

    
for i,(train_idx,val_idx) in enumerate(kf.split(X)):    

    X_train, y_train = X.iloc[train_idx,:], y_hat.iloc[train_idx]

    X_val, y_val = X.iloc[val_idx, :], y_hat.iloc[val_idx]
    

    print('\nFold: {}\n'.format(i+1))
    #12,0.8,1000
    lg=LGBMClassifier(boosting_type='gbdt',learning_rate=0.08,depth=12,objective='multiclass',n_estimators=1000,num_class=11,
                     metric='multi_error',colsample_bytree=0.5,reg_alpha=2,reg_lambda=2,random_state=294,n_jobs=-1)

#     X_train,y_train=SMOTETomek(random_state=294).fit_resample(X_train,y_train)
    lg.fit(X_train,y_train)

    print(accuracy_score(y_val,lg.predict(X_val)))

    acc_score+=accuracy_score(y_val,lg.predict(X_val))
    
    preds[i+1]=lg.predict(X_main_test)